In [1]:
import pandas as pd
import os
from datetime import datetime

In [26]:
dtypes = {
    'ride_id': str,
    'rideable_type': str,
    'started_at': str,
    'ended_at': str,
    'start_station_name': str,
    'start_station_id': str,
    'end_station_name': str,
    'end_station_id': str,
    'start_lat': float,
    'start_lng': float,
    'end_lat': float,
    'end_lng': float,
    'member_casual': str,
    'time': float,
    'distance': float,
}

In [27]:
data_path = os.getcwd() + '/../data/data_dist_time.csv'
data = pd.read_csv(data_path, dtype=dtypes, index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/pollyren/Desktop/stuff/divvy/eda/../data/data_dist_time.csv'

In [19]:
data['started_at'] = pd.to_datetime(data['started_at'])
data['ended_at'] = pd.to_datetime(data['ended_at'])

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16031963 entries, 0 to 16048416
Data columns (total 15 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    object        
 6   end_station_name    object        
 7   end_station_id      object        
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
 13  time                object        
 14  distance            object        
dtypes: datetime64[ns](2), float64(4), object(9)
memory usage: 1.9+ GB


In [21]:
data.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,time,distance
0,A847FADBBC638E45,docked_bike,2020-04-26 17:45:14,2020-04-26 18:12:03,Eckhart Park,86,Lincoln Ave & Diversey Pkwy,152,41.8964,-87.6610,41.9322,-87.6586,member,1609.0,2.4738936122865414
1,5405B80E996FF60D,docked_bike,2020-04-17 17:08:54,2020-04-17 17:17:03,Drake Ave & Fullerton Ave,503,Kosciuszko Park,499,41.9244,-87.7154,41.9306,-87.7238,member,489.0,0.608713436919493
2,5DD24A79A4E006F4,docked_bike,2020-04-01 17:54:13,2020-04-01 18:08:36,McClurg Ct & Erie St,142,Indiana Ave & Roosevelt Rd,255,41.8945,-87.6179,41.8679,-87.6230,member,863.0,1.8545824831649016
3,2A59BBDF5CDBA725,docked_bike,2020-04-07 12:50:19,2020-04-07 13:02:31,California Ave & Division St,216,Wood St & Augusta Blvd,657,41.9030,-87.6975,41.8992,-87.6722,member,732.0,1.3305885086688019
4,27AD306C119C6158,docked_bike,2020-04-18 10:22:59,2020-04-18 11:15:54,Rush St & Hubbard St,125,Sheridan Rd & Lawrence Ave,323,41.8902,-87.6262,41.9695,-87.6547,casual,3175.0,5.666709461947201


In [ ]:
data.rideable_type.unique()
docked = data[data['rideable_type'] == 'docked_bike']
electric = data[data['rideable_type'] == 'electric_bike']
classic = data[data['rideable_type'] == 'classic_bike']

In [22]:
zero = data[data['started_at'].dt.year == 2020]
one = data[data['started_at'].dt.year == 2021]
two = data[data['started_at'].dt.year == 2022]